In [205]:
values = [{'customer_id': 215016144, 'order_date': '2019-03-28', 'order_value': 2701}, {'customer_id': 751835361, 'order_date': '2019-03-29', 'order_value': 7435}, {'customer_id': 795037452, 'order_date': '2019-03-30', 'order_value': 8001}, {'customer_id': 595278971, 'order_date': '2019-03-31', 'order_value': 8912}, {'customer_id': 718888189, 'order_date': '2019-04-01', 'order_value': 3635}, {'customer_id': 644263827, 'order_date': '2019-04-02', 'order_value': 1978}, {'customer_id': 584725362, 'order_date': '2019-04-03', 'order_value': 9847}, {'customer_id': 980721773, 'order_date': '2019-04-04', 'order_value': 9322}, {'customer_id': 179384062, 'order_date': '2019-04-05', 'order_value': 3052}, 
          {'customer_id': 215016144, 'order_date': '2018-03-28', 'order_value': 1000},
         {'customer_id': 215016144, 'order_date': '2017-03-28', 'order_value': 1000}]

In [206]:
import pandas as pd
df = pd.DataFrame(list(values))
df_date = df[["customer_id", "order_date"]]

df["order_date_min"] = df["order_date"]
df["avg_monetary"] = df["order_value"]
df["frequency"] = df["customer_id"]
df.head()

,customer_id,order_date,order_value,order_date_min,avg_monetary,frequency
0,215016144,2019-03-28,2701,2019-03-28,2701,215016144
1,751835361,2019-03-29,7435,2019-03-29,7435,751835361
2,795037452,2019-03-30,8001,2019-03-30,8001,795037452
3,595278971,2019-03-31,8912,2019-03-31,8912,595278971
4,718888189,2019-04-01,3635,2019-04-01,3635,718888189


In [207]:
df = (df.groupby("customer_id").agg({"order_value": "sum", 
                                    "order_date": "max",
                                    "order_date_min": "min",
                                    "avg_monetary": "mean",
                                   "frequency": "count"})
      .reset_index()
      .rename(columns={'order_value':'monetary'}))


df.head()
df[df["customer_id"] == 215016144]

,customer_id,monetary,order_date_min,order_date,frequency,avg_monetary
1,215016144,4701,2017-03-28,2019-03-28,3,1567


In [208]:
import datetime as dt
df["today"] = dt.date.today()
df["recency"] = pd.to_datetime(df["order_date"]) - pd.to_datetime(df["today"])
df["first_purchase"] = pd.to_datetime(df["order_date_min"]) - pd.to_datetime(df["today"])
df[df["customer_id"] == 215016144]

,customer_id,monetary,order_date_min,order_date,frequency,avg_monetary,today,recency,first_purchase
1,215016144,4701,2017-03-28,2019-03-28,3,1567,2019-11-10,-227 days,-957 days


In [209]:
df["recency"] = df.apply(lambda x: x["recency"].days, axis=1)
df["first_purchase"] = df.apply(lambda x: x["first_purchase"].days, axis=1)

In [210]:
df.head()

,customer_id,monetary,order_date_min,order_date,frequency,avg_monetary,today,recency,first_purchase
0,179384062,3052,2019-04-05,2019-04-05,1,3052,2019-11-10,-219,-219
1,215016144,4701,2017-03-28,2019-03-28,3,1567,2019-11-10,-227,-957
2,584725362,9847,2019-04-03,2019-04-03,1,9847,2019-11-10,-221,-221
3,595278971,8912,2019-03-31,2019-03-31,1,8912,2019-11-10,-224,-224
4,644263827,1978,2019-04-02,2019-04-02,1,1978,2019-11-10,-222,-222


In [211]:
df_date.sort_values("customer_id")[df_date["customer_id"] == 215016144]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,customer_id,order_date
0,215016144,2019-03-28
9,215016144,2018-03-28
10,215016144,2017-03-28


In [212]:
import numpy as np

df_date["order_date"] = pd.to_datetime(df_date["order_date"])


new_df = pd.DataFrame({"customer_id":[], 
                    "diff_date":[]})
for customer_id in df_date.groupby("customer_id")["customer_id"]:
    filter_values = df_date[df_date["customer_id"] == customer_id[0]].sort_values(["customer_id", "order_date"])["order_date"].diff(periods=-1)
    temp_df = pd.DataFrame({"customer_id":customer_id[0], 
                    "diff_date":filter_values})
    new_df = new_df.append(temp_df)
    
new_df["diff_date"] = new_df.apply(lambda x: x["diff_date"].days, axis=1)
new_df["customer_id"] = new_df["customer_id"].astype(int)
new_df["std_dev"] = new_df["diff_date"]


new_df = new_df.groupby("customer_id").agg({"diff_date": "mean",
                                            "std_dev": np.std})

In [216]:
new_df.head()

,diff_date,std_dev
customer_id,,
179384062,NaN,NaN
215016144,-365.0,0.0
584725362,NaN,NaN
595278971,NaN,NaN
644263827,NaN,NaN


In [214]:
df_merged = pd.merge(df, new_df, on=["customer_id", "customer_id"])

In [220]:
df_merged.head()

,customer_id,monetary,order_date_min,order_date,frequency,avg_monetary,today,recency,first_purchase,diff_date,std_dev
0,179384062,3052,2019-04-05,2019-04-05,1,3052,2019-11-10,-219,-219,NaN,NaN
1,215016144,4701,2017-03-28,2019-03-28,3,1567,2019-11-10,-227,-957,-365.0,0.0
2,584725362,9847,2019-04-03,2019-04-03,1,9847,2019-11-10,-221,-221,NaN,NaN
3,595278971,8912,2019-03-31,2019-03-31,1,8912,2019-11-10,-224,-224,NaN,NaN
4,644263827,1978,2019-04-02,2019-04-02,1,1978,2019-11-10,-222,-222,NaN,NaN


In [219]:
values = []
for value in df_merged.values:
    values.append({"customer_id": value[0], 
    "monetary": value[1],
    "order_date_min": value[2],
    "order_date": value[3],
    "frequency": value[4],
    "avg_monetary": value[5],
    "recency": value[7],
    "first_purchase": value[8],
    "avg_date": value[9],
    "std_dev_date": value[10]})

In [299]:
df_score_frequency = df[["customer_id", "frequency", ]]
df_score_monetary = df[["customer_id", "monetary", ]]

In [301]:
df_score_frequency = df_score_frequency.sort_values("frequency", ascending=False).rolling(min_periods=1, 
                                                                    window=df_score_frequency["frequency"].count(),
                                                                    on="customer_id").sum()

df_score_frequency["customer_id"] = df_score_frequency["customer_id"].astype(int)

last_value = df_score_frequency["frequency"].iloc[-1]
df_score_frequency["per_frequency"] = df_score_frequency["frequency"].apply(lambda x: (x/last_value))

def score(value):
    if value < .2:
        return 1
    elif value < .4:
        return 2
    elif value < .6:
        return 3
    elif value < .8:
        return 4
    else:
        return 5

df_score_frequency["score_frequency"] = df_score_frequency["per_frequency"].apply(lambda x: score(x))

In [302]:
df_score_frequency.sort_values("frequency")

,customer_id,frequency,per_frequency,score_frequency
1,215016144,3.0,0.272727,2
0,179384062,4.0,0.363636,2
2,584725362,5.0,0.454545,3
3,595278971,6.0,0.545455,3
4,644263827,7.0,0.636364,4
5,718888189,8.0,0.727273,4
6,751835361,9.0,0.818182,5
7,795037452,10.0,0.909091,5
8,980721773,11.0,1.000000,5


In [303]:
df_score_frequency

,customer_id,frequency,per_frequency,score_frequency
1,215016144,3.0,0.272727,2
0,179384062,4.0,0.363636,2
2,584725362,5.0,0.454545,3
3,595278971,6.0,0.545455,3
4,644263827,7.0,0.636364,4
5,718888189,8.0,0.727273,4
6,751835361,9.0,0.818182,5
7,795037452,10.0,0.909091,5
8,980721773,11.0,1.000000,5


In [307]:
df_score_frequency[df_score_frequency["score_frequency"] == 3]["frequency"]

2    5.0
3    6.0
Name: frequency, dtype: float64

In [324]:
df_merged.to_sql()

TypeError: to_sql() missing 2 required positional arguments: 'name' and 'con'